In [497]:
from PIL import Image
import numpy as np
import math
import sys, pygame
import matplotlib.pyplot as plt
import time
PI = 3.1415926535
import pygame.math as py_math

In [519]:
class Lidar():
    def __init__(self, R, angle):
        self.R = R 
        self.sensor_data = math.inf
        self.position = [0,0]
        self.angle = angle
        
    def sense(self, drone_position, angle):
        point_to_check = []
        for i in range(1, self.R + 1):
            point = [0,0]
            check = self.change_position(i, angle)
            point[1] = check[1] + drone_position[1]
            point[0] = check[0] + drone_position[0]
            point_to_check.append(point)
        return point_to_check
    
    def change_position(self, R, angle):
        line_end_position = [0,0]
        line_end_position[1] = round(R * math.cos(angle * PI / 180))
        line_end_position[0] = round(R * math.sin(angle * PI / 180))
        return line_end_position

In [526]:
class Drone:
    def __init__(self, x, y, angle, lidars):
        self.position = [0,0]
        self.angle = angle
        self.lidars = lidars
        self.speed = 0
        self.speed_in_ms = 0
        self.yaw_speed = 1
        self.acc = 0
        
    def init_lidars(self):
        for lidar in self.lidars:
            tmp = self.change_position(lidar.R, lidar.angle)
            lidar.position[1] = self.position[1] + tmp[1]
            lidar.position[0] = self.position[0] + tmp[0]
            
    def move(self):
        self.check_angle()
        tmp = self.change_position(3, self.angle)
        self.position[1] += self.speed * tmp[1]
        self.position[0] += self.speed * tmp[0]
        for lidar in self.lidars:
            lidar.position[1] += self.speed * tmp[1]
            lidar.position[0] += self.speed * tmp[0]

        
    def change_position(self, R, angle):
        line_end_position = [0,0]
        line_end_position[1] = round((R * math.cos(angle * PI / 180)))
        line_end_position[0] = round((R * math.sin(angle * PI / 180)))
        return line_end_position
    
    def change_angle(self, direction):
        self.angle += self.yaw_speed * direction
        self.check_angle()
        for lidar in self.lidars:   
            tmp = self.change_position(lidar.R, self.angle + lidar.angle)
            lidar.position[1] = self.position[1] + tmp[1]
            lidar.position[0] = self.position[0] + tmp[0]
        
    
    def check_angle(self):
        if self.angle >= 360:
            self.angle = 0
        if self.angle < 0:
            self.angle = 359
        
    

In [527]:
class Simulator:
    def __init__(self, map_path, drone, scale):
        self.map_arr = self.png_to_arr(map_path)
        self.screen = self.init_pygame(self.map_arr.shape[0], self.map_arr.shape[1])
        self.drone = drone
        self.founed_points = []
        self.time_in_sec = 0
        self.flying = True
        self.scale = scale
        
    def png_to_arr(self, map_path):
        img = Image.open(map_path)
        img=img.convert('1')
        return np.flip(np.rot90(np.asarray(img)),axis=0)
    
    def init_pygame(self, width, height):
        pygame.init()
        size = width, height
        screen = pygame.display.set_mode(size)
        return screen
    
    def render_map(self):
        for x in range(self.map_arr.shape[0]):
            for y in range(self.map_arr.shape[1]):
                if self.map_arr[x][y] == False:
                    pygame.draw.circle(self.screen,[255,0,0],[x,y],1,1)
    
    def draw_drone(self):
        self.screen.fill([255,255,255])
        pygame.draw.circle(self.screen, [255,0,0], self.drone.position, 3,3)
        for lidar in self.drone.lidars:            
            pygame.draw.line(self.screen, [0,0,0], self.drone.position , lidar.position)
            pygame.draw.circle(self.screen, [255,0,0], lidar.position, 1,1) 
            
    def find_start_pos(self):
        start_flag = False
        for y in range(self.map_arr.shape[0]):
            for x in range(self.map_arr.shape[1]):
                if self.map_arr[x,y] == True:
                    for lidar in self.drone.lidars:            
                        lidar.position = [x+5, y + 60 + 5]
                    return [x+5, y+5]
    
    def check_points(self, sensor_data, lidar):
        for point in sensor_data:      
            if self.map_arr[point[0],point[1]] == False:
                self.founed_points.append([point[0],point[1]])
                self.print_founded_points()
                lidar.sensor_data = math.hypot(point[1] - self.drone.position[1], point[0] - self.drone.position[0])
                return
        lidar.sensor_data = math.inf
                
    def print_founded_points(self):
        for edge in self.founed_points:
            pygame.draw.circle(self.screen,[255,0,0],edge,1,1)
    
    def render_map(self):
        for x in range(self.map_arr.shape[0]):
            for y in range(self.map_arr.shape[1]):
                if self.map_arr[x][y] == False:
                    pygame.draw.circle(self.screen,[255,0,0],[x,y],1,1)
    
    def crash(self):
        if not self.map_arr[self.drone.position[0]][self.drone.position[1]]:
            print("You Crashed!")
            self.flying = False
         
    def draw_param(self, position):
        pygame.font.init()
        myfont = pygame.font.SysFont('Comic Sans MS', 30)
        textsurface = myfont.render(self.build_param(position), False, (0, 0, 0))
        self.screen.blit(textsurface,(0,0))
        
    def build_lidars_param(self):
        i = 1
        ans = ""
        for lidar in self.drone.lidars:
            ans += "Lidar " + str(i) + ")" + str("{:.1f}".format(lidar.sensor_data*self.scale)) + " "
            i += 1
        return ans
    
    def calc_speed(self, position):
        return math.hypot(position[1] - self.drone.position[1], position[0] - self.drone.position[0])
        
    def build_param(self,position):
        return "Speed:" + str("{:.2f}".format(self.drone.speed_in_ms * self.scale / 100)) +'|' \
                "X:" + str(self.drone.position[0])+'|' \
                "Y:" + str(self.drone.position[1])+'|' \
                "GYRO:" + str(self.drone.angle) +'|' \
                "LIDARS:" + self.build_lidars_param() + "TIME:" + str("{:.1f}".format(self.time_in_sec))+'|' 
        
    def block(self):
        t_end = time.time() + 0.1
        while time.time() < t_end:
            pass     
        self.time_in_sec += 0.1
    
    def is_full_second(self, time_in_sec):
        return time_in_sec - math.floor(time_in_sec) < 0.1
    
    def simulate(self):
        self.drone.position = self.find_start_pos()
        pos = self.find_start_pos()
        self.drone.init_lidars()
        while self.flying:
            if self.is_full_second(self.time_in_sec):
                self.drone.speed_in_ms = self.calc_speed(pos)
                pos[0] = self.drone.position[0]
                pos[1] = self.drone.position[1]
            self.block()
            self.draw_drone()
            self.draw_param(pos)
            for lidar in self.drone.lidars:
                point_to_check = lidar.sense(self.drone.position, lidar.angle + self.drone.angle)
                self.check_points(point_to_check, lidar)
            self.print_founded_points()
            pygame.display.flip()                
            self.drone.move()
            self.crash()
            for event in pygame.event.get(): 
                if event.type==pygame.KEYDOWN:
                    if event.key==pygame.K_UP:
                        self.drone.speed += 1 if self.drone.speed < 3 else 0
                        
                    if event.key==pygame.K_DOWN:
                        self.drone.speed -= 2
                        if self.drone.speed <= 0:
                            self.drone.acc = 0
                            self.drone.speed = 0
                            
                    if event.key==pygame.K_r:
                        t_end = time.time() + 5
                        while time.time() < t_end:
                            self.render_map()
                            pygame.display.flip()
                    
                    if event.key==pygame.K_ESCAPE:
                        pygame.quit()
                        flag = False
            keys_pressed = pygame.key.get_pressed()

            if keys_pressed[pygame.K_LEFT]:
                self.drone.change_angle(-1)
                for lidar in self.drone.lidars:
                    point_to_check = lidar.sense(self.drone.position, lidar.angle + self.drone.angle)
                    self.check_points(point_to_check, lidar)
                    self.print_founded_points()
                    
            if keys_pressed[pygame.K_RIGHT]:
                self.drone.change_angle(1)
                for lidar in self.drone.lidars:
                    point_to_check = lidar.sense(self.drone.position, lidar.angle + self.drone.angle)
                    self.check_points(point_to_check, lidar)
                    self.print_founded_points()
                    
        pygame.quit()